# Code for labeling (not training) all data given model weights

### Note
There is an error where the prediction function will be incorrect if there is a single item in the final batch. We have been fixing this manually because of the few times we had to run the notebook, so keep this error in mind.

In [1]:
import torch
import pickle
import regex as re


import os
import random as rand
from tqdm import tqdm, trange

import numpy as np
import pandas as pd
# from clean_text import stopwords_make, punctstr_make, unicode_make, apache_tokenize, clean_sentence_apache 

import matplotlib.pyplot as plt


from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import RandomSampler, SequentialSampler
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import transformers
import sys, argparse
import numpy as np
import re
import random
from tqdm import tqdm
from collections import Counter
from transformers import BertTokenizer, BertModel
from collections import Counter

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from transformers import AdamW
from transformers import get_scheduler

from transformers import LongformerModel, LongformerTokenizer


Bad key savefig.frameon in file C:\Users\theth\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\theth\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\theth\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probably need to get an updated matplotlibrc fil

In [2]:
model_location = 'model_orgs.bin'
test_files = 'filtered_preprocessed_texts_65365_110521.pkl'

In [3]:
def open_test_data(path):
    return open(path, 'rb')
with open_test_data(test_files) as f:
    cult = pickle.load(f)


In [4]:
cult

,file_name,text
0,/vol_b/data/jstor_data/ocr/journal-article-10....,"[[Research, Note, Church, Membership, in, The,..."
1,/vol_b/data/jstor_data/ocr/journal-article-10....,"[[polish, (io),oo, sociological, review, ISSN,..."
2,/vol_b/data/jstor_data/ocr/journal-article-10....,"[[Article, ■jjDlBSj, grapliy, Compassionate, a..."
3,/vol_b/data/jstor_data/ocr/journal-article-10....,"[[REPLY, TO, ALLISON:, MORE, ON, COMPARING, RE..."
4,/vol_b/data/jstor_data/ocr/journal-article-10....,"[[Determinants, of, Spousal, Interaction:, Mar..."
...,...,...
65360,/vol_b/data/jstor_data/ocr/journal-article-10....,"[[JOURNAL, OF, MANAGERIAL, ISSUES, Vol., XXIII..."
65361,/vol_b/data/jstor_data/ocr/journal-article-10....,"[[Social, Psychology, Quarterly, Vol., No., Tr..."
65362,/vol_b/data/jstor_data/ocr/journal-article-10....,"[[COMPULSORY, CLOSETS, AND, THE, SOCIAL, CONTE..."
65363,/vol_b/data/jstor_data/ocr/journal-article-10....,"[[mir, vol., mir, IfflullQQOITIOiii, Internoti..."


In [5]:
import itertools
full_text = []

for i in cult['text']:
    joined = list(itertools.chain(*i))
    full_text.append(" ".join(joined))

In [6]:
cult['full_text'] = full_text

In [7]:
def remove_tags(article):
    article = re.sub('<plain_text> <page sequence="1">', '', article)
    article = re.sub(r'</page>(\<.*?\>)', ' \n ', article)
    # xml tags
    article = re.sub(r'<.*?>', '', article)
    article = re.sub(r'<body.*\n\s*.*\s*.*>', '', article)
    return article

tags_removed = [remove_tags(art) for art in cult['full_text']]
cult['text_no_tags'] = tags_removed

In [8]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1070 with Max-Q Design


In [9]:
class BERTClassifier(nn.Module):

        
    def __init__(self, params):
        super().__init__()
        
        if 'dropout' in params:
            self.dropout = nn.Dropout(p=params['dropout'])
        else:
            self.dropout = None
            
#         self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False, do_basic_tokenize=False)
#         self.bert = BertModel.from_pretrained("bert-base-uncased")

        self.max_length = params['max_length'] if 'max_length' in params else 768
        
        self.tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
        self.bert = LongformerModel.from_pretrained("allenai/longformer-base-4096", gradient_checkpointing=True)

        self.num_labels = params["label_length"]

        self.fc = nn.Linear(768, self.num_labels)

    def get_batches(self, all_x, all_y, batch_size=10):

        """ Get batches for input x, y data, with data tokenized according to the BERT tokenizer 
        (and limited to a maximum number of WordPiece tokens """

        batches_x=[]
        batches_y=[]

        for i in range(0, len(all_x), batch_size):

            current_batch=[]

            x=all_x[i:i+batch_size]

            batch_x = self.tokenizer(x, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
            batch_y=all_y[i:i+batch_size]

            batches_x.append(batch_x.to(device))
            batches_y.append(torch.LongTensor(batch_y).to(device))

        return batches_x, batches_y


    def forward(self, batch_x): 

        bert_output = self.bert(input_ids=batch_x["input_ids"],
                         attention_mask=batch_x["attention_mask"],
#                          token_type_ids=batch_x["token_type_ids"],
                         output_hidden_states=True)

        # We're going to represent an entire document just by its [CLS] embedding (at position 0)
        # And use the *last* layer output (layer -1)
        # as a result of this choice, this embedding will be optimized for this purpose during the training process.

        bert_hidden_states = bert_output['hidden_states']

        out = bert_hidden_states[-1][:,0,:]
        
        if self.dropout:
            out = self.dropout(out)

        out = self.fc(out)

        return out.squeeze()

    def evaluate(self, batch_x, batch_y):

        self.eval()
        corr = 0.
        total = 0.

        with torch.no_grad():
            for x, y in zip(batch_x, batch_y):
                y_preds = self.forward(x)
                for idx, y_pred in enumerate(y_preds):
                    prediction=torch.argmax(y_pred)
                    if prediction == y[idx]:
                        corr += 1.
                    total+=1                          
        return corr/total



In [10]:
def get_preds(model, batch_x, batch_y):
    """
    given a certain model, outputs predictions for the batch_x
    Note that batch_y contains dummy labels to take advantage of pre-existing batching code
    """

    model.eval()
    
    predictions = []
    scores = []

    with torch.no_grad():
        for x, y in tqdm(zip(batch_x, batch_y)):
            y_preds = model.forward(x)
            for idx, y_pred in enumerate(y_preds):
                predictions.append(torch.argmax(y_pred).item())
                sm = F.softmax(y_pred, dim=0)
                try:
                    scores.append(sm[1].item())
                except:
                    
                    print(len(scores))
                    print(sm.shape, sm)
    return predictions, scores

In [11]:
model = BERTClassifier({'label_length': 2})
if torch.cuda.is_available():    
    
    model.load_state_dict(torch.load(model_location))
    model = model.to(device)
# If not...
else:
    model.load_state_dict(torch.load(model_location, map_location=torch.device('cpu')))

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
X_train1 = cult['text_no_tags']
y_train1 = np.zeros(len(X_train1))

In [13]:
batch_x, batch_y = model.get_batches(list(X_train1), list(y_train1), batch_size=2)

In [14]:
preds, scores = get_preds(model, batch_x, batch_y)

32683it [5:51:45,  1.55it/s]

65364
torch.Size([]) tensor(1., device='cuda:0')
65364
torch.Size([]) tensor(1., device='cuda:0')


In [27]:
# with open(f'{model_location}_pred.pkl', 'wb') as f:
#     pickle.dump(preds, f)

# with open(f'{model_location}_score.pkl', 'wb') as f:
#     pickle.dump(scores, f)

In [17]:
# with open(f'{model_location}_pred.pkl', 'rb') as f:
#     preds = pickle.load(f)

# with open(f'{model_location}_score.pkl', 'rb') as f:
#     scores = pickle.load(f)

In [18]:
print(preds[:10])
print(scores[:10])

[0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[0.1411495953798294, 0.5261998772621155, 0.0008700518519617617, 0.0008788488921709359, 0.030665069818496704, 0.016456294804811478, 0.012148287147283554, 0.00021816176013089716, 0.08609135448932648, 0.9997815489768982]


In [18]:
for pred, score in zip(preds, scores):
    if pred == 1:
        assert score >= 0.5
    else:
        assert score <= 0.5

In [21]:
len(preds)

65366

In [19]:
out = model.forward(batch_x[-1])
F.softmax(out, dim=0)

tensor([ 2.5446, -1.8247], device='cuda:0', grad_fn=<SqueezeBackward0>)

In [20]:
preds[65364] = 0

In [23]:
scores[65364] = 0.0125

In [30]:
with open(f'{model_location}_pred.pkl', 'wb') as f:
    pickle.dump(preds, f)

with open(f'{model_location}_score.pkl', 'wb') as f:
    pickle.dump(scores, f)

[0.010806157253682613,
 0.0004782340838573873,
 0.7749702334403992,
 0.00039812311297282577,
 0.9839450716972351,
 0.000224292729399167,
 0.011638615280389786,
 0.11250873655080795,
 0.013985664583742619,
 0.99974]